In [6]:
import numpy as np


print(np.__version__)
print("if this^ is >=2 then pymatching might fail")

1.26.4
if this^ is >=2 then pymatching might fail


In [7]:
# reload magic 
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import stim
import pymatching

In [10]:
L = 3
xs, zs, Hx, Hz = generate_rotated_surface_code_stabilizers(L)
print("xs", xs)
print("zs", zs)


xs [stim.PauliString("+XX_XX____"), stim.PauliString("+____XX_XX"), stim.PauliString("+__X__X___"), stim.PauliString("+___X__X__")]
zs [stim.PauliString("+___ZZ_ZZ_"), stim.PauliString("+_ZZ_ZZ___"), stim.PauliString("+ZZ_______"), stim.PauliString("+_______ZZ")]


In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import hstack, kron, eye, csc_matrix, block_diag


def repetition_code(n):
    """
    Parity check matrix of a repetition code with length n.
    """
    row_ind, col_ind = zip(*((i, j) for i in range(n) for j in (i, (i+1)%n)))
    data = np.ones(2*n, dtype=np.uint8)
    return csc_matrix((data, (row_ind, col_ind)))


def toric_code_x_stabilisers(L):
    """
    Sparse check matrix for the X stabilisers of a toric code with
    lattice size L, constructed as the hypergraph product of
    two repetition codes.
    """
    Hr = repetition_code(L)
    H = hstack(
            [kron(Hr, eye(Hr.shape[1])), kron(eye(Hr.shape[0]), Hr.T)],
            dtype=np.uint8
        )
    H.data = H.data % 2
    H.eliminate_zeros()
    return csc_matrix(H)



In [18]:
from scipy.sparse import hstack, kron, eye, csc_matrix, block_diag
def toric_code_x_logicals(L):
    """
    The logical operator is X...X either horizontally or vertically across the lattice, in the unrotated version.
    In the rotated version 
    """
    if L != 3:
        raise NotImplementedError("Only L=3 is implemented")
    vertical_x = [1]*L + [0]*L*(L - 1)
    _x = ([1] + [0]*(L-1)) * L
    return np.array([
        [0, 0, 0, 1, 1, 1, 0, 0, 0], # center horizontal.
        # [1, 0, 0, 0, 1, 0, 0, 0, 1], # diagonal from top left to bottom right
    ])



In [21]:
("""
            
         0 --- 1 --- 2
         |  X  |     | X
         3 --- 4 --- 5
       X |     |  X  |        <--- qubit numbering and X stabilizers 
         6 --- 7 --- 8
                  
    
""")
# OKAY IM A FUCKING IDIOT, WE NEED TO BE NUMBERING THE SQUARES, NOT THE QUBITS!
# see fig 5: https://arxiv.org/pdf/2307.14989
from pymatching import Matching
# generate a matching object specific to the PCM for x-stabilizers
matching_x = Matching(Hx)
print("PCM for x-stabilizers, L=3 rotated surface code")
print(Hx)
# matching = Matching.from_check_matrix(Hx, use_virtual_boundary_node=True)
matching = Matching(Hx)
print(matching)
for noise_i in range(9):
  noise = np.zeros(9, dtype=int)
  noise[noise_i] = 1
  print("z noise:", noise)
  s = Hx @ noise % 2
  print("syndrome:", s)
  c = matching.decode(s)
  print("predicted physical error:", c)

  x_logicals = toric_code_x_logicals(L)
  logical_flip_pred = x_logicals @ c % 2
  logical_flip_true = x_logicals @ noise % 2
  print("true logical error:", logical_flip_true)
  print("predicted logical error:", logical_flip_pred)
  print()

PCM for x-stabilizers, L=3 rotated surface code
[[1 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 1]
 [0 0 1 0 0 1 0 0 0]
 [0 0 0 1 0 0 1 0 0]]
<pymatching.Matching object with 4 detectors, 1 boundary node, and 7 edges>
z noise: [1 0 0 0 0 0 0 0 0]
syndrome: [1 0 0 0]
predicted physical error: [1 0 0 0 0 0 0 0 0]
true logical error: [0]
predicted logical error: [0]

z noise: [0 1 0 0 0 0 0 0 0]
syndrome: [1 0 0 0]
predicted physical error: [1 0 0 0 0 0 0 0 0]
true logical error: [0]
predicted logical error: [0]

z noise: [0 0 1 0 0 0 0 0 0]
syndrome: [0 0 1 0]
predicted physical error: [0 0 1 0 0 0 0 0 0]
true logical error: [0]
predicted logical error: [0]

z noise: [0 0 0 1 0 0 0 0 0]
syndrome: [1 0 0 1]
predicted physical error: [0 0 0 1 0 0 0 0 0]
true logical error: [1]
predicted logical error: [1]

z noise: [0 0 0 0 1 0 0 0 0]
syndrome: [1 1 0 0]
predicted physical error: [0 0 0 0 1 0 0 0 0]
true logical error: [1]
predicted logical error: [1]

z noise: [0 0 0 0 0 1 0 0 0]
syndrome: [0 1 1 

In [135]:
from pymatching import Matching
# generate a matching object specific to the PCM for x-stabilizers
matching_x = Matching(Hx)

# The noise we care about deepends _only_ on the z-type pauilis occuring the error string
# i.e. if an error E = [ e_x e_z ] in binary symplectic form, then Hx only 'sees' e_z
p = 0.1
# for my project, this will just go over all possible errors :)
# z_noise = np.random.binomial(1, p, size=Hx.shape[1])
z_noise = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0])
z_syndrome = Hx @ z_noise % 2
print("Z noise:", z_noise)

# generate a prediction and predict logical error
prediction = matching_x.decode(z_syndrome)
x_logicals = toric_code_x_logicals(L)
logical_error_pred = x_logicals @ prediction % 2
logical_error_true = x_logicals @ z_noise % 2

print("Prediction: ", prediction)
print("True logical obs flipped: ", logical_error_true)
print("Predicted logical obs flipped: ", logical_error_pred)


Z noise: [0 0 0 1 0 0 0 0 0]
Prediction:  [0 0 0 0 0 0 0 0 0]
True logical obs flipped:  [1 0]
Predicted logical obs flipped:  [0 0]


8 9 1


Maximum likelihood, L=3
0.9397690759087137


In [11]:
from itertools import chain, combinations
from functools import reduce
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
        


In [15]:
enumerate(powerset([1,2,3]))

#### what we need is:
1. answers for MLE on every syndrome -> separates good and bad
2. for all "good" syndromes, we need answers from pymatching.

#### build the parity check matrices

so MWPM decoding for X-type and Z-type errors is the same process (PCM might be different), so our procedure is:
1. get the syndrome logical pair $(\sigma, \ell)$
2. Check that $f^*(\sigma) = \ell$, so we know we have a good example
3. Let $\sigma = [\sigma_X | \sigma_Z]$
4. MWPM decode $\sigma$ into $\ell_X \ell_Z = \hat{\ell}$, our guess for the logical operator
5. if $\hat{\ell} \notin \ell \mathcal{S}$, then we have an important example

Issues:
 - organizing everything might be tough; do we start with $\ell$
 - idea: I build the prism from the ground up; I could run MWPM at each layer before generating $\ell \mathcal{S}$, then check equality as I fill out that coset
 - probably need to downgrade to numpy 1.26

In [19]:
import numpy as np
from scipy.sparse import hstack, kron, eye, csc_matrix, block_diag


def repetition_code(n):
    """
    Parity check matrix of a repetition code with length n.
    """
    row_ind, col_ind = zip(*((i, j) for i in range(n) for j in (i, (i+1)%n)))
    data = np.ones(2*n, dtype=np.uint8)
    return csc_matrix((data, (row_ind, col_ind)))


def toric_code_x_stabilisers(L):
    """
    Sparse check matrix for the X stabilisers of a toric code with
    lattice size L, constructed as the hypergraph product of
    two repetition codes.
    """
    Hr = repetition_code(L)
    H = hstack(
            [kron(Hr, eye(Hr.shape[1])), kron(eye(Hr.shape[0]), Hr.T)],
            dtype=np.uint8
        )
    H.data = H.data % 2
    H.eliminate_zeros()
    return csc_matrix(H)



In [ ]:
toric_code